In [1]:
import csv
import pandas as pd
import numpy as np

import sys
DROPBOX='/Users/hongjian/Dropbox (CGC)/'
sys.path.append(DROPBOX+'analysis/info/')
sys.path.append(DROPBOX+'analysis/utils/')
from info import *
from utils import *

from matrix_score import matrix_score
pd.options.display.max_columns = 120
%matplotlib inline

In [2]:
def sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value):
    """ function that selected colums from wgsa, add some other feathers 
        set . into 0
    """
    with open(fin) as f:
        r = csv.reader(f)
        head = r.next()
        feat = wgsa_feat + add_feat + extra_feat
        for line in r:
            info = dict(zip(head, line))
            aaref, aaalt, aapos = info['aaref'], info['aaalt'], info['aapos']
            
            
            # no aaalt 
            if aaref is not "U" and aaalt is not ".":
                # reformat wsga feat, missing value filled with 0, will -1 be better?
                for c in wgsa_feat:
                    if info[c] == '.':
                        info[c] = -1
                    else:
                        info[c] = float(info[c])
                # set some default value for extra_feat
                for c in extra_feat:
                    info[c] = info.get(c, 'NA')
                
                info['genename'] = gene_trans(info['genename'])
                info['blosum62'] = matrix_score(aaref, aaalt, 'blosum62')
                info['pam250'] = matrix_score(aaref, aaalt, 'pam250')
                
                info['target'] = target_value
                if target_value == 'NA' and 'category' in info:
                    if info['category'] == 'TP':
                        info['target'] = 1
                    elif info['category'] == 'TN':
                        info['target'] = 0
                        
                
                # update SUMO/phospho scores
                info['phospho_score'] = 0
                info['phospho_cutoff'] = 0
                info['phospho_diff'] = 0
                gene = info['genename']
                if gene in phosphorylation:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in phosphorylation[gene]:
                            if phosphorylation[gene][pos]['AA'] == aaref:
                                info['phospho_score'] = phosphorylation[gene][pos]['Score']
                                info['phospho_cutoff'] = phosphorylation[gene][pos]['Cutoff']
                                info['phospho_diff'] = phosphorylation[gene][pos]['diff']
                                break
                                
                info['SUMO_score'] = 0
                info['SUMO_cutoff'] = 0
                info['SUMO_diff'] = 0
                if gene in SUMO:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # pos is 1 based
                        if pos in SUMO[gene]:
                            if SUMO[gene][pos]['AA'] == aaref:
                                info['SUMO_score'] = SUMO[gene][pos]['Score']
                                info['SUMO_cutoff'] = SUMO[gene][pos]['Cutoff']
                                info['SUMO_diff'] = SUMO[gene][pos]['diff']
                                break  
                                
                # add inteface flag
                info['interface'] = 0
                if gene in interface:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in interface[gene]:
                                info['interface'] = 1
                                
                # add ASA score Accessible Surface Areas
                info['ASA'] = 0
                if gene in ASA:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ASA[gene]:
                                info['ASA'] = ASA[gene][pos]
                
                info['secondary_H'] = 0
                info['secondary_C'] = 0
                info['secondary_E'] = 0
                if gene in secondary:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in secondary[gene]:
                                if secondary[gene][pos] == 'H':
                                    info['secondary_H'] = 1
                                elif secondary[gene][pos] == 'C':
                                    info['secondary_C'] = 1    
                                elif secondary[gene][pos] == 'E':
                                    info['secondary_E'] = 1
                                    
                info['ubiquitination'] = 0
                if gene in ubiquitination:
                    aapos = info['aapos'].split(';')
                    for pos in aapos:
                        pos = int(pos)
                        # AA_seq start from 0
                        protein_length = len(AA_seq[gene])
                        if pos < protein_length and AA_seq[gene][pos-1] == aaref:
                            if pos in ubiquitination[gene]:
                                info['ubiquitination'] = ubiquitination[gene][pos]
                
                # gene specific feathers
                info['complex_CORUM'] = 0
                if gene in complex_CORUM:
                    info['complex_CORUM'] = 1
                
                info['preppi_counts'] = 0
                if gene in preppi:
                    info['preppi_counts'] = preppi[gene]
                    
                info['s_hat'] = 0
                if gene in s_hat:
                    info['s_hat'] = s_hat[gene]
                    
                info['pli'] = pli.get(info['genename'], '0')
                info['lofz'] = lofz.get(info['genename'], '0')
#                 if float(info['pli']) >= 0.5: # HS genes
#                 #if float(info['pli']) < 0.5: # HIS genes
#                     continue
                
                w.writerow([info[c] for c in feat])


In [3]:
# Load protein related annotations
SUMO = np.load('../data/protein/SUMO.npy').item()
phosphorylation = np.load('../data/protein/phosphorylation.npy').item()
AA_seq = np.load('../data/protein/AA_seq.npy').item()
interface = np.load('../data/protein/interface.npy').item()
ASA = np.load('../data/protein/ASA.npy').item()
preppi = np.load('../data/protein/preppi.npy').item()
secondary = np.load('../data/protein/secondary.npy').item()
ubiquitination = np.load('../data/protein/ubiquitination.npy').item()

complex_CORUM = set()
with open('../data/protein/protein_complex_CORUM.txt') as f:
    for line in f:
        lst = line.strip().split('\t')
        complex_CORUM = complex_CORUM | set(lst)
s_hat = {}
with open('../data/gene_s_het.txt') as f:
    head = f.readline()
    for line in f:
        lst = line.strip().split('\t')
        s_hat[lst[0]] = float(lst[1])

In [4]:
# add feathers from WGSA and other inputs, some of them need to be excluded in future training
rank_score_cols = ['SIFT_converted_rankscore', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HVAR_rankscore', 
 'LRT_converted_rankscore', 'MutationTaster_converted_rankscore', 'MutationAssessor_score_rankscore', 
 'FATHMM_converted_rankscore', 'PROVEAN_converted_rankscore', 'VEST3_rankscore', 
 'MetaSVM_rankscore', 'MetaLR_rankscore', 'M-CAP_rankscore', 
 'CADD_raw_rankscore', 'DANN_rankscore', 'fathmm-MKL_coding_rankscore', 
 'Eigen-PC-raw_rankscore', 'GenoCanyon_score_rankscore', 'integrated_fitCons_score_rankscore', 
 'GM12878_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 
 'HUVEC_fitCons_score_rankscore', 'GERP++_RS_rankscore', 
 'phyloP100way_vertebrate_rankscore', 'phyloP20way_mammalian_rankscore', 
 'phastCons100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 
 'SiPhy_29way_logOdds_rankscore']
wgsa_feat = ['1000Gp3_AF', 'ExAC_AF', 'CADD_phred', 'Eigen-phred', 'Eigen-PC-phred']
wgsa_feat = wgsa_feat + rank_score_cols

add_feat =  ['blosum62', 'pam250', 'SUMO_score', 'SUMO_cutoff', 'SUMO_diff',
             'phospho_score', 'phospho_cutoff','phospho_diff', 'interface',
             'ASA', 'pli', 'lofz', 'complex_CORUM', 'preppi_counts',
             'secondary_H', 'secondary_C', 'secondary_E', 'ubiquitination',
             's_hat',
             'target']

# feathers used for future info
extra_feat = ['#chr', 'pos(1-based)',  'ref', 'alt', 'category', 'source','INFO', 'disease', 'genename']

In [5]:
outname = '../data/input_data.csv'

with open(outname, 'w') as fw:
    w = csv.writer(fw)
    w.writerow(wgsa_feat + add_feat + extra_feat)

    # positive training
    fin = '../data/HGMD_DM_rare_missense.anno.rare.csv' 
    target_value = 1
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)

    # negative data
    fin = '../data/DiscovEHR_rare_missense_30000.csv' 
    target_value = 0
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)
    
    # metaSVM training 
    fin = '../data/metaSVM/metaSVM_train.anno.rare.csv' 
    target_value = 'NA'
    sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)

In [6]:
# does this have to be <1 %?? 

fins = ['../data/metaSVM/metaSVM_train.anno.rare.csv', '../data/metaSVM/metaSVM_test1.anno.rare.csv', 
       '../data/metaSVM/metaSVM_test2.anno.rare.csv', '../data/metaSVM/metaSVM_test3.anno.rare.csv', 
       '../data/metaSVM/metaSVM_addtest1.anno.rare.csv', '../data/metaSVM/metaSVM_addtest2.anno.rare.csv']
fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + '.reformat.csv')
for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        w.writerow(wgsa_feat + add_feat + extra_feat)
        target_value = 'NA'
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)

In [7]:
fins = ['../data/ExAC_nonTCGA_missense_all.anno.rare_30000.csv', 
        '../data/case_control/control_900.anno.rare.csv',
        '../data/case_control/control_1911.anno.rare.csv']

fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + '.reformat.csv')

for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        w.writerow(wgsa_feat + add_feat + extra_feat)

        target_value = 0
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)

In [8]:
fins = ['../data/case_control/case.anno.rare.csv', 
        '../data/cancer_hotspots/hotspot.anno.rare.csv',
       '../data/case_control/DDD_new_0.2.anno.rare.csv']

fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + '.reformat.csv')

for fin, fout in zip(fins, fouts):
    with open(fout, 'w') as fw:
        w = csv.writer(fw)
        w.writerow(wgsa_feat + add_feat + extra_feat)

        target_value = 1
        sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value)